# Introduction

In this notebook, we experiment with various compression methods for postings lists. 


In [1]:
from math import log

DEBUG = True
#DEBUG = False

def p(msg):
    if DEBUG:
        print('.. {}'.format(msg))

def ilog2(x):
    return int(log(x, 2.0))

def encode_unary(vals):
    out = ""

    vs = [vals] if type(vals) == int else vals
    for v in vs:
        out = out + "1" * v
        out = out + "0"

    return out

def encode_binary(vals, width):
    out = ""

    vs = [vals] if type(vals) == int else vals
    for v in vs:
        for i in range(width, 0, -1):
            bit = v >> (i-1) & 0x0001
            if bit > 0:
                out = out + "1"
            else:
              out = out + "0"

    return out


# should check if input val is within the coding range (e.g., >0)
def encode_gamma(vals):
    out_code = []
    for v in vals:
        k_d = ilog2(v)
        k_r = v - (1 << k_d)
        p('val = {}, k_d = {}, k_r = {}'.format(v, k_d, k_r))
        out1 = encode_unary(k_d)
        out2 = encode_binary(k_r, k_d)
        out_code.append([out1, out2])
    return out_code
def encode_delta(vals):
    out_code = []
    for v in vals:
        k_d = ilog2(v)
        k_r = v - (1 << k_d)
        p('val = {}, k_d = {}, k_r = {}'.format(v, k_d, k_r))
        out1 = len(bin(v)[2:])
        prefix = encode_gamma([out1])
        out2 = encode_binary(k_r, k_d)
        out_code.append(prefix[0]+[out2])
    return out_code
# out_list is a list of encodings for each value, where each encoding is a list of component
def pp_binary(out_list, width=8, as_string=True):
    if as_string:
        # print as strings of width-byte chunks
        # first, flatten the list
        l = ["".join(v) for v in out_list]
        str = "".join(l)
        n = len(str)
        s = 0
        while s < n:
            e = min(s + width, n)
            print("{} ".format(str[s:e]), end="")
            s += width
        print()
    else:
        for component in out_list:
            print(component)




In [2]:
def test_gamma(values):
    out = encode_gamma(values)
    #print(out)
    pp_binary(out, as_string=False)
#     pp_binary(out)
def test_delta(values):
    out = encode_delta(values)
    print(out)
    pp_binary(out, as_string=False)

In [3]:
print([x for x in range(2,16)])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [4]:
test_gamma([511])

.. val = 511, k_d = 8, k_r = 255
['111111110', '11111111']


In [8]:
test_delta([255])

.. val = 255, k_d = 7, k_r = 127
.. val = 8, k_d = 3, k_r = 0
[['1110', '000', '1111111']]
['1110', '000', '1111111']


In [6]:
# test_gamma([16,255,1023])

## Exercise

1. Implement a method `decode_gamma` that takes a gamma-encoded binary string and decode it into a list of integers. 
2. Implement the encoding and decoding methods for delta, rice, and variable byte methods. 